In [1]:
%pip install langchain
%pip install openai
%pip install PyPDF2
%pip install faiss-cpu
%pip install tiktoken
%pip install PyDrive

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from PyPDF2 import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import ElasticVectorSearch, Pinecone, Weaviate, FAISS

In [3]:
# Get your API keys from openai, you will need to create an account.
# Here is the link to get the keys: https://platform.openai.com/account/billing/overview
import os
os.environ["OPENAI_API_KEY"] = "{ADD KEY HERE}"

In [32]:
# Specify the path to the directory containing your PDF files
directory_path = '{Path/To/Directory}'

# List all PDF files in the directory
pdf_files = [file for file in os.listdir(directory_path) if file.endswith('.pdf')]

# Loop through the list of PDF files and read each one
for pdf_file in pdf_files:
    pdf_file_path = os.path.join(directory_path, pdf_file)  # Full path to the PDF file
    pdf_reader = PdfReader(pdf_file_path)

    # Access the number of pages in the PDF
    num_pages = len(pdf_reader.pages)
    print(f'Total number of pages in {pdf_file}: {num_pages}')
    raw_text = ''
    # Access the text content of each page
    for page_num, page in enumerate(pdf_reader.pages):
            text = page.extract_text()
            if text:
                raw_text += text

Total number of pages in Draft DE-LC-000L110 FY24 TCF Base Joint Lab Call_11.01.2023.pdf: 109


In [33]:
raw_text[:100]

' \nQUESTIONS ABOUT THIS LAB CALL? EMAIL TCF@ HQ.DOE.GOV. \nPROBLEMS WITH EXCHANGE ? EMAIL EERE- EXCHAN'

In [34]:
# We need to split the text that we read into smaller chunks so that during information retreival we don't hit the token size limits.

# lines = (line.strip() for line in raw_text.splitlines())
# chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
# text = "\n".join(chunk for chunk in chunks if chunk)

text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 5000,
    chunk_overlap  = 650,
    length_function = len,
)
texts = text_splitter.split_text(raw_text)

In [36]:
len(texts)

61

In [37]:
texts[0]

'QUESTIONS ABOUT THIS LAB CALL? EMAIL TCF@ HQ.DOE.GOV. \nPROBLEMS WITH EXCHANGE ? EMAIL EERE- EXCHANGE SUPPORT @HQ.DOE.GOV & \nINCLUDE LAB CALL NAME AND NUMBER IN SUBJECT LINE . \n \nU.S. Department of Energy  \nOffice of Technology Transitions  \nTechnology Commercialization  Fund  \nBase Annual Appropriations  \n \nDRAFT National Laboratory  Call for Proposals  \nCore Laboratory Infrastructure for M arket Readiness  (CLIMR ) \n \nDE-LC-000L 110 \n \nFiscal Year 202 4 \n \n \n \n \n \n \nThis lab call is being issued as part of the Technology Commercialization Fund Base Annual \nAppropriations by the U.S. Department of Energy’s  (DOE’s) Office of Technology Transitions  \nthe Office of Cybersecurity, Energy Security, and Emergency Response, Office of Electricity, \nOffice of Fossil Energy and Carbon Management, Office of Nuclear Energy, and the Office of \nEnergy Efficiency and Renewable Energy, in particular: Advanced Material s and Manufacturing \nTechnologies Office, Bioenergy Tech

In [41]:
# Download embeddings from OpenAI
embeddings = OpenAIEmbeddings()

In [42]:
docsearch = FAISS.from_texts(texts, embeddings)

In [13]:
docsearch

In [43]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

In [44]:
chain = load_qa_chain(OpenAI(), chain_type="stuff")

In [45]:
import openai
# import docsearch
query = "I am a small biotech startup that wants to partner with a DOE lab for an experiment I focus on fermentation based biomanufacturing what are relevant topics for my company in this lab call"

response = openai.ChatCompletion.create(
  model="gpt-4-1106-preview", 
      messages=[
        {"role": "system", "content": "You are a helpful Biotechnology assistant that generates questions."},
        {"role": "user", "content": f"Generate 3 detailed questions based off of: {query}"}
    ],
  max_tokens=200,
  temperature=0.6
),
# print(response)
# print(response[0]["choices"])
# print(response[0]["choices"][0]["message"])
# print(response[0]["choices"][0]["message"]["content"])
doc_chunks = []
for question in response[0]["choices"][0]["message"]["content"].split('\n'):
  print(question)
  docs = docsearch.similarity_search(question, k=2)
  doc_chunks.append(docs)
# docs = docsearch.similarity_search(query)
# chain.run(input_documents=docs, question=query)
summaries = []  
for chunk in doc_chunks:
    # print(chunk)
    response = openai.ChatCompletion.create(
      model="gpt-4-1106-preview",
      messages=[
            {
                "role": "system",
                "content": "You are a specialized AI Biotechnology Analyst. Your primary mission is to conduct in-depth research and analysis in the field of biotechnology. Your expertise spans a wide range of biotechnological domains, including genetic engineering, bioprocessing, bioinformatics, and molecular biology."
            },
            {
                "role": "user",
                "content": f"Read this information and summarize it in a few sentences: {chunk[0].page_content}"
            }
        ],
      max_tokens=800,
      temperature=0.3
    )
    summaries.append(response['choices'][0]['message']['content'])

print(summaries)


1. What specific capabilities and resources does the DOE lab offer that can enhance the scalability and efficiency of fermentation-based biomanufacturing processes within a small biotech startup environment? Please provide detailed information on equipment, expertise, and any unique analytical methods that could support the development and optimization of microbial strains, fermentation conditions, or downstream processing techniques.

2. How can a small biotech startup like ours engage in a collaborative partnership with a DOE lab to leverage high-throughput screening and omics technologies for the improvement of fermentation-based biomanufacturing? Can you outline the process for accessing these technologies, including any requirements for data sharing, intellectual property considerations, and the potential for co-publishing research findings?

3. Considering our focus on fermentation-based biomanufacturing, what opportunities are available through the DOE lab for piloting novel fee

In [46]:
response = openai.ChatCompletion.create(
      model="gpt-4-1106-preview",
      messages=[
            {
                "role": "system",
                "content": "You are a specialized AI Biotechnology Analyst. Your primary mission is to conduct in-depth research and analysis in the field of biotechnology. Your expertise spans a wide range of biotechnological domains, including genetic engineering, bioprocessing, bioinformatics, and molecular biology."
            },
            {
                "role": "user",
                "content": f'Read this information {summaries} Using the above information, answer the following'\
           f' question or topic: "{question}" in a detailed report --'\
           " The report should focus on the answer to the question, should be well structured, informative," \
           " in depth, with facts and numbers if available, a minimum of 1,200 words and with markdown syntax and apa format. "\
            "You MUST determine your own concrete and valid opinion based on the given information. Do NOT deter to general and meaningless conclusions." \
            }
        ],
      max_tokens=1500,
      temperature=0.4
    )
print(response['choices'][0]['message']['content'])

# Opportunities for Fermentation-Based Biomanufacturing Through DOE Labs

## Introduction

Fermentation-based biomanufacturing is a critical process in the biotechnology industry, enabling the conversion of various feedstocks into valuable products such as biofuels, bioplastics, and pharmaceuticals. The Department of Energy (DOE) has a vested interest in advancing these technologies, particularly as they pertain to sustainable energy and material production. This report examines the opportunities available through DOE labs for piloting novel feedstock utilization strategies, optimizing energy consumption, and reducing waste in fermentation-based processes. It also explores past successful partnerships and projects that have led to the commercialization of sustainable biotechnologies.

## DOE's Commitment to Biomanufacturing Innovation

The DOE's Bioenergy Technologies Office (BETO) and various initiatives under its umbrella demonstrate a strong commitment to supporting the development 

In [50]:
query = "I have a small startup in the biotech space. We do fermentation based biomanufacturing of chemicals usually created with petrol using biomass. What are relevant Areas of Interest for grant proposals in this lab call"
docs = docsearch.similarity_search(query)

summaries = []  

# print(chunk)
response = openai.ChatCompletion.create(
    model="gpt-4-1106-preview",
    messages=[
        {
            "role": "system",
            "content": "You are a specialized AI Biotechnology Analyst. Your primary mission is to conduct in-depth research and analysis in the field of biotechnology. Your expertise spans a wide range of biotechnological domains, including genetic engineering, bioprocessing, bioinformatics, and molecular biology."
        },
        {
            "role": "user",
            "content": f"Read this information and summarize it in a few sentences: {docs[0].page_content}"
        }
    ],
    max_tokens=800,
    temperature=0.3
)
summaries.append(response['choices'][0]['message']['content'])

print(summaries)

response = openai.ChatCompletion.create(
      model="gpt-4-1106-preview",
      messages=[
            {
                "role": "system",
                "content": "You are a specialized AI Biotechnology Analyst. Your primary mission is to conduct in-depth research and analysis in the field of biotechnology. Your expertise spans a wide range of biotechnological domains, including genetic engineering, bioprocessing, bioinformatics, and molecular biology."
            },
            {
                "role": "user",
                "content": f'Read this information {summaries} Using the above information, answer the following'\
           f' question or topic: "{query}" in a detailed report --'\
           " The report should focus on the answer to the question, should be well structured, informative," \
           " in depth, with facts and numbers if available, a minimum of 1,200 words and with markdown syntax and apa format. "\
            "You MUST determine your own concrete and valid opinion based on the given information. Do NOT deter to general and meaningless conclusions." \
            }
        ],
      max_tokens=1500,
      temperature=0.4
    )
print(response['choices'][0]['message']['content'])

['The provided text outlines a lab call for proposals related to the development of algae-based wastewater treatment as part of broader biotechnological research and development efforts. The focus is on mid-stage R&D to create cost-effective conversion technologies for low-carbon bioenergy and bioproducts from various feedstocks and waste, including innovations to reduce cost and carbon intensity in converting feedstock into intermediate and final products. It also emphasizes the need for advancements in systems development and integration to reduce risks and produce biofuels and bioproducts that meet distribution and end-use specifications. Additionally, the text highlights the importance of data, modeling, and analysis to track progress and assess the impacts of advanced bioenergy systems.\n\nThere are two main technology areas for proposals: \n\n1. Early Maturation of Technologies (Technology Area #1): This area focuses on DOE Facility-developed technologies with commercial promise 

In [ ]:
query = "How was the model trained?"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' The model was trained with LoRA (Hu et al., 2021) on the 437,605 post-processed examples for four epochs. Detailed model hyper-parameters and training code can be found in the associated repository and model training log.'

In [ ]:
query = "what was the size of the training dataset?"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' The final training dataset contains 437,605 prompt-generation pairs.'